In [ ]:
!pip install -q kaggle
!mkdir ~/ .kaggle

mkdir: cannot create directory ‘/root/’: File exists


In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gopalbhattrai/pascal-voc-2012-dataset

100% 3.50G/3.52G [00:29<00:00, 211MB/s]
100% 3.52G/3.52G [00:32<00:00, 118MB/s]


In [ ]:
import tensorflow as tf
import numpy as np
import os
import shutil as sh

In [ ]:
!unzip "/content/pascal-voc-2012-dataset.zip" -d "dataset"

Streaming output truncated to the last 5000 lines.
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001874.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001876.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001882.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001885.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001895.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001896.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001926.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001966.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001971.png  
  inflating: dataset/VOC2012_train_val/VOC2012_train_val/SegmentationClass/2008_001992.png  
  inflating: datase

In [ ]:
train_images='/content/dataset/VOC2012/JPEGImages/'
train_maps='/content/dataset/VOC2012/Annotations/'

val_images='/content/dataset/VOC2012/ValJPEGImages/'
val_maps='/content/dataset/VOC2012/ValAnnotations/'

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
N_CLASSES=len(classes)
H,W =224,224
SPLIT_SIZE=H//32
print(SPLIT_SIZE)
N_EPOCHS=135
BATCH_SIZE=32

7


In [ ]:
with open("dataset/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000027.xml") as f:
  print(f.read())

<annotation>
	<folder>VOC2012</folder>
	<filename>2007_000027.jpg</filename>
	<source>
		<database>The VOC2007 Database</database>
		<annotation>PASCAL VOC2007</annotation>
		<image>flickr</image>
	</source>
	<size>
		<width>486</width>
		<height>500</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>person</name>
		<pose>Unspecified</pose>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>174</xmin>
			<ymin>101</ymin>
			<xmax>349</xmax>
			<ymax>351</ymax>
		</bndbox>
		<part>
			<name>head</name>
			<bndbox>
				<xmin>169</xmin>
				<ymin>104</ymin>
				<xmax>209</xmax>
				<ymax>146</ymax>
			</bndbox>
		</part>
		<part>
			<name>hand</name>
			<bndbox>
				<xmin>278</xmin>
				<ymin>210</ymin>
				<xmax>297</xmax>
				<ymax>233</ymax>
			</bndbox>
		</part>
		<part>
			<name>foot</name>
			<bndbox>
				<xmin>273</xmin>
				<ymin>333</ymin>
				<xmax>297</xmax>
				<ymax>354</ymax>
			</bndbox>
		</part>
		<part>
			<name>foot<

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
preprocess_xml("/content/dataset/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000027.xml")

NameError: ignored

In [ ]:
def preprocess_xml(filename):
  print("inside the preprocess_xml func")
  tree=ET.parse(filename)
  root=tree.getroot()
  size_tree=root.find("size")
  height=float(size_tree.find("height").text)
  width=float(size_tree.find("width").text)
  # print(height,width)
  for object_tree in root.findall("object"):
    for b_box in object_tree.iter("bndbox"):
      x_min=float(b_box.find("xmin").text)
      x_max=float(b_box.find("xmax").text)
      y_min=float(b_box.find("ymin").text)
      y_max=float(b_box.find("ymax").text)
      # print(x_min,x_max,y_min,y_max)
      break
    b_boxes=[]
    class_name=object_tree.find("name").text
    class_dict={cls:ind for ind,cls in enumerate(classes)}
    bounding_box=[
        (x_min+x_max)/(2*width),(y_min+y_max)/(2*height),(x_max-x_min)/width,
        (y_max-y_min)/height,class_dict[class_name]
    ]
    b_boxes.append(np.array(bounding_box))

  return tf.convert_to_tensor(b_boxes,dtype=tf.float32)


In [ ]:
b_boxes=preprocess_xml("/content/dataset/VOC2012_train_val/VOC2012_train_val/Annotations/2007_000027.xml")

inside the preprocess_xml func


In [ ]:
# b_boxes

In [ ]:
def generate_output(b_boxs):
  out_labels=np.array(np.zeros([SPLIT_SIZE,SPLIT_SIZE,5+N_CLASSES]),dtype="float32")
  for b in range(len(b_boxs)):
    grid_x=b_boxs[...,b,0]*SPLIT_SIZE
    grid_y=b_boxs[...,b,1]*SPLIT_SIZE
    i=int(grid_x)
    j=int(grid_y)
    if (out_labels[i,j,0]==0):
      print(i,j)
      out_labels[i,j,0:5]=((1.,grid_x%1,grid_y%1,b_boxs[...,b,2],b_boxs[...,b,3]))
      out_labels[i,j,5+int(b_boxs[...,b,4])]=1.
  return tf.convert_to_tensor(out_labels,dtype=tf.float32)


In [ ]:
out=generate_output(b_boxes)

3 3


In [ ]:
out[...,1,0]

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
(0.5380658436213992*7)%1,(0.452*7)%1

(0.7664609053497946, 0.16400000000000015)

In [ ]:
int(0.7664609053497946),int(0.16400000000000015)

(0, 0)

In [ ]:
out[0][3,3]

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [ ]:
# tf.math.argmax(out[0][3,3])

In [ ]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [ ]:
pth_jpeg=os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val","JPEGImages")
pth_annot=os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val","Annotations")

In [ ]:
val_img=os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val","val_images")
val_anno=os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val","val_annot")
os.makedirs(val_img,exist_ok=True)
os.makedirs(val_anno,exist_ok=True)

In [ ]:
os.path.splitext("test.txt")[0]


'test'

In [ ]:
train_img=[os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val/JPEGImages",fn)
   for fn in os.listdir("/content/dataset/VOC2012_train_val/VOC2012_train_val/JPEGImages")]

train_annot=[ os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val/Annotations",fn)
    for fn in os.listdir("/content/dataset/VOC2012_train_val/VOC2012_train_val/Annotations")]

val_img=[ os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val/val_images",fn)
   for fn in  os.listdir("/content/dataset/VOC2012_train_val/VOC2012_train_val/val_images")]

val_annot=[ os.path.join("/content/dataset/VOC2012_train_val/VOC2012_train_val/val_annot",fn)
   for fn in  os.listdir("/content/dataset/VOC2012_train_val/VOC2012_train_val/val_annot")]

In [ ]:
# for fn in val_list:
#   sh.move(os.path.join(pth_jpeg,fn),val_img)
#   afn=os.path.splitext(fn)[0]+".xml"
#   sh.move(os.path.join(pth_annot,afn),val_anno)

In [ ]:
train_data=tf.data.Dataset.from_tensor_slices((train_img,train_annot))
val_data=tf.data.Dataset.from_tensor_slices((val_img,val_annot))

In [ ]:
list(train_data.take(1))

[(<tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012_train_val/VOC2012_train_val/JPEGImages/2009_004037.jpg'>,
  <tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012_train_val/VOC2012_train_val/Annotations/2010_003341.xml'>)]

In [ ]:
def get_bboxes(fn_img,fn_anno):
  # print(fn_img.numpy())
  img_arr=tf.io.decode_jpeg(tf.io.read_file(fn_img))
  img=tf.cast(tf.image.resize(img_arr,(H,W)),dtype=tf.float32)
  b_boxes=tf.numpy_function(func=preprocess_xml,inp=[fn_anno],Tout=tf.float32)
  return img,b_boxes

In [ ]:
# tf.io.decode_jpeg(tf.io.read_file(val_img[0]))

In [ ]:
# img_arr=tf.io.decode_jpeg(tf.io.read_file(fn_img))

In [ ]:
val_data

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [ ]:
train_data=train_data.map(get_bboxes)

In [ ]:
val_data=val_data.map(get_bboxes)

TypeError: ignored

In [ ]:
for i,j in train_data.take(1):
  print(i)

inside the preprocess_xml func
tf.Tensor(
[[[1.7675797e+02 1.8930165e+02 1.6838298e+02]
  [1.7839574e+02 1.8951498e+02 1.7345537e+02]
  [1.7944165e+02 1.9144165e+02 1.7144165e+02]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

 [[1.7602678e+02 1.8969930e+02 1.6843750e+02]
  [1.7664255e+02 1.8982112e+02 1.7282112e+02]
  [1.7917857e+02 1.9217857e+02 1.7417857e+02]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]]

 [[1.7460779e+02 1.8960779e+02 1.6976851e+02]
  [1.7584280e+02 1.8984280e+02 1.7284280e+02]
  [1.7814891e+02 1.9307748e+02 1.7411319e+02]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00]
  [9.6428585e-01 9.6428585e-01 9.6428585e-01]
  [9.6428585e-01 9.6428585e-01 9.6428585e-01]]

 ...

 [[1.9394094e+01 2.8063122e+01 3.9917111e+00]
  [2.1116079e+01 2.7344339e+01 1.2783492e+00]
  [2.124

In [ ]:
def preprocess(img,bb_boxs):
  gen_out=tf.numpy_function(func=generate_output,inp=[bb_boxs],Tout=tf.float32)
  return img,gen_out

In [ ]:
train_data=train_data.map(preprocess)
# val_data=val_data.map(preprocess)

In [ ]:
train_data.as_numpy_iterator().next()

In [ ]:
train_data=train_data.batch(32).prefetch(tf.data.AUTOTUNE)
# val_data=val_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
# train_data.take(1).as_numpy_iterator().next()

In [ ]:
#base_model = tf.keras.applications.resnet50.ResNet50(
base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False

27018416/27018416 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.layers import Conv2D,BatchNormalization,LeakyReLU,Flatten,Dense,Dropout,GlobalAvgPool2D,Reshape
NUM_FILTERS=512/2
OUTPUT_DIM=N_CLASSES+5*B

model=tf.keras.Sequential([
  base_model,
  Conv2D(NUM_FILTERS,(3,3), padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  LeakyReLU(alpha=0.1),

  GlobalAvgPool2D(),

  Dense(NUM_FILTERS,kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Dropout(0.5),

  Dense(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIM,activation='sigmoid'),

  Reshape((SPLIT_SIZE,SPLIT_SIZE,OUTPUT_DIM)),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional)  (None, 7, 7, 1280)       6575239   
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 256)         2949376   
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 256)        1024      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 7, 256)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 256)         590080    
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 256)        1024      
 hNormalization)                                        

In [ ]:
some=tf.random.normal((32,224,224,3))

In [ ]:
x=tf.constant([[[2,2,3,4,4],[2,3,2,3,2]]])

In [ ]:
x

<tf.Tensor: shape=(1, 2, 5), dtype=int32, numpy=
array([[[2, 2, 3, 4, 4],
        [2, 3, 2, 3, 2]]], dtype=int32)>

In [ ]:
x[:]

<tf.Tensor: shape=(1, 2, 5), dtype=int32, numpy=
array([[[2, 2, 3, 4, 4],
        [2, 3, 2, 3, 2]]], dtype=int32)>

In [ ]:
tf.where(x[:]==4)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[0, 0, 3],
       [0, 0, 4]])>

In [ ]:
tf.gather_nd(x,tf.where(x[:]==4))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 4], dtype=int32)>

In [ ]:
def yolo_loss(y_true,y_pre):
  target=y_true[...,0]
  y_true_extract=tf.gather_nd(y_true,tf.where(target[:]==1.))
  y_pre_extract=tf.gather_nd(y_pre,tf.where(target[:]==1.))

  y_pre_extract

In [ ]:
import tensorflow as tf

In [ ]:
def res_layer(x):
